In [8]:
import os
import selenium.webdriver
import csv
import time
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options  # for suppressing the browser
from concurrent.futures import ThreadPoolExecutor, wait

In [9]:
url1= "https://walmart.com/ip"

url_sets=["https://www.walmart.com/browse/personal-care/electric-toothbrushes/1005862_1007221_2709578_4710",
         "https://www.walmart.com/browse/personal-care/razors/1005862_1007220_1007399",
         "https://www.walmart.com/browse/personal-care/deodorants-antiperspirants/1005862_1001487",
         "https://www.walmart.com/browse/personal-care/body-wash-cleansers/1005862_1071969_1074364",
         "https://www.walmart.com/browse/health/multivitamins/976760_1005863_1699667",
         "https://www.walmart.com/browse/health/mood-stress-support-supplements/976760_1005863_9579127",
         "https://www.walmart.com/browse/health/health-monitors/976760_1005860_1161882",
         "https://www.walmart.com/browse/protein-fitness/976760_1166769"]

categories=["Oral","Razors","Deos","Bath","Vitamins",
           "supplements","Health_monitors","Proteins"]

In [12]:
def run_process(page_number,url_sets):
    for i in range(len(url_sets)):
        top_n= ["1","2","3","4","5","6","7","8","9","10"]
        url_cat=url_sets[i]+"?page="+top_n[page_number]
        option = webdriver.ChromeOptions()
        option.add_argument('headless')
        driver= webdriver.Chrome(executable_path='/Users/bhaskar/Documents/chromedriver',options=option)
        driver.get(url_cat)
        body_cat = driver.find_element_by_tag_name("body").get_attribute("innerHTML")
        driver.quit()
        soupBody_cat = BeautifulSoup(body_cat)
        for tmp in soupBody_cat.find_all('div', {'class':'search-result-gridview-item-wrapper'}):

            final_results.append(tmp['data-id'])

In [13]:

futures = []
final_results = []
with ThreadPoolExecutor() as executor:
    for number in range(10):
        futures.append(
            executor.submit(run_process,number,url_sets)
        )
wait(futures)

DoneAndNotDoneFutures(done={<Future at 0x7f8d62a0d250 state=finished returned NoneType>, <Future at 0x7f8d62a06e50 state=finished returned NoneType>, <Future at 0x7f8d62a0da90 state=finished returned NoneType>, <Future at 0x7f8d62a3b090 state=finished returned NoneType>, <Future at 0x7f8d62a3b0d0 state=finished returned NoneType>, <Future at 0x7f8d629afd10 state=finished returned NoneType>, <Future at 0x7f8d62a43510 state=finished returned NoneType>, <Future at 0x7f8d62a0dd50 state=finished returned NoneType>, <Future at 0x7f8d62954b50 state=finished returned NoneType>, <Future at 0x7f8d62a3bfd0 state=finished returned NoneType>}, not_done=set())

In [14]:
len(final_results)

2472

top_n= ["1","2","3","4","5","6","7","8","9","10"]
final_results = []
for i in range(len(top_n)):
    url_cat="https://www.walmart.com/browse/electronics/all-laptop-computers/3944_3951_1089430_132960"+"?page="+top_n[i]
    option = webdriver.ChromeOptions()
    option.add_argument('headless')
    driver= webdriver.Chrome(executable_path='C:/chromedriver.exe',options=option)
    driver.get(url_cat)
    body_cat = driver.find_element_by_tag_name("body").get_attribute("innerHTML")
    driver.quit()
    soupBody_cat = BeautifulSoup(body_cat)
    for tmp in soupBody_cat.find_all('div', {'class':'search-result-gridview-item-wrapper'}):

        final_results.append(tmp['data-id'])

In [15]:
len(final_results)

codelist=final_results

In [16]:
codelist[0]

'2YT8ZV7Y1Z49'

In [17]:
def run_process2(i,codelist):
    #creating a list without the place taken in the first loop
    print(i)
    item_wlmt=codelist[i]
    url2=url1+"/"+item_wlmt
    print(url2)
    option = webdriver.ChromeOptions()
    option.add_argument('headless')

    try:
        driver= webdriver.Chrome(executable_path='/Users/bhaskar/Documents/chromedriver',options=option) # Chrome driver is being used.
        print ("Requesting URL: " + url2)
        driver.get(url2)   # URL requested in browser.
        print ("Webpage found ...")
        time.sleep(5)
        # Find the document body and get its inner HTML for processing in BeautifulSoup parser.
        body = driver.find_element_by_tag_name("body").get_attribute("innerHTML")
        print("Closing Chrome ...") # No more usage needed.
        driver.quit() 				# Browser Closed.

        print("Getting data from DOM ...")
        soupBody = BeautifulSoup(body) # Parse the inner HTML using BeautifulSoup
        #print(soupBody)

        h1ProductName = soupBody.find("h1", {"class": "prod-ProductTitle prod-productTitle-buyBox font-bold"})
        divProductDesc = soupBody.find("div", {"class": "about-desc about-product-description xs-margin-top"})
        liProductBreadcrumb_parent = soupBody.find("li", {"data-automation-id": "breadcrumb-item-0"})
        liProductBreadcrumb_active = soupBody.find("li", {"class": "breadcrumb active"})
        spanProductPrice = soupBody.find("span", {"class": "price-group"})
        spanProductRating = soupBody.find("span", {"itemprop": "ratingValue"})
        spanProductRating_count = soupBody.find("span", {"class": "stars-reviews-count-node"})
            
        ################# exceptions #########################
        if divProductDesc is None:
            divProductDesc="Not Available"
        else:
            divProductDesc=divProductDesc
                
        if liProductBreadcrumb_parent is None:
            liProductBreadcrumb_parent="Not Available"
        else:
            liProductBreadcrumb_parent=liProductBreadcrumb_parent
                
        if liProductBreadcrumb_active is None:
            liProductBreadcrumb_active="Not Available"
        else:
            liProductBreadcrumb_active=liProductBreadcrumb_active
                
        if spanProductPrice is None:
            spanProductPrice="NA"
        else:
            spanProductPrice=spanProductPrice

        if spanProductRating is None or spanProductRating_count is None:
            spanProductRating=0.0
            spanProductRating_count="0 ratings"
            #print(spanProductBreadcrumb_active.text)
        else:
            spanProductRating=spanProductRating.text
            spanProductRating_count=spanProductRating_count.text


            ### Recommended Products
        reco_prods=[]
        for tmp in soupBody.find_all('a', {'class':'tile-link-overlay u-focusTile'}):
            reco_prods.append(tmp['data-product-id'])

        if len(reco_prods)==0:
            reco_prods=["Not available"]
        else:
            reco_prods=reco_prods
        print(len(reco_prods))
        WLMTData.append([codelist[i],h1ProductName.text, divProductDesc.text,url2,
                         liProductBreadcrumb_parent.text,liProductBreadcrumb_active.text,
                         spanProductPrice.text,spanProductRating,spanProductRating_count,reco_prods])
    except Exception as e:
        print (str(e))

In [18]:
futures2 = []
WLMTData = [["Product_code","Product_name", "Product_description","Product_URL",
                 "Breadcrumb_parent","Breadcrumb_active",
                "Product_price", "Rating_Value","Rating_Count","Recommended_Prods"]]
with ThreadPoolExecutor() as executor:
    for number in range(len(codelist)):
        futures2.append(
            executor.submit(run_process2,number,codelist)
        )
wait(futures2)

01
https://walmart.com/ip/2YT8ZV7Y1Z49

https://walmart.com/ip/1I8FI39V72PV
2
https://walmart.com/ip/6XG88B1NZMNB
3
https://walmart.com/ip/47XLQASR4C88
45
https://walmart.com/ip/0U3YGN4RJB8T

https://walmart.com/ip/62NAL3OAYMDD
6
https://walmart.com/ip/2TN0IC0PAINM7
8

https://walmart.com/ip/6MXK34G8P79H
https://walmart.com/ip/31SXJANX5JBP
9
https://walmart.com/ip/4E0MA760QN5V
1011

https://walmart.com/ip/1PB0ZJHYIJ40
https://walmart.com/ip/53V4NM0KP2DK
12
https://walmart.com/ip/23QVUVRLJDV1
13
https://walmart.com/ip/12JB22DMYZWG
14
https://walmart.com/ip/6LU4EVHGE95S
1516
https://walmart.com/ip/3DDRU7CJC46T

https://walmart.com/ip/3ZE3CO8UX0CJ
17
https://walmart.com/ip/5TDIJMGBSZUV
18
https://walmart.com/ip/2C28YUPHV3UN
19
https://walmart.com/ip/1SLP0ZRCUWSH
20
https://walmart.com/ip/56UQIZX6X0PS
21
22https://walmart.com/ip/7ERUOBKQF075
23

https://walmart.com/ip/4XPVRY0B04R9
https://walmart.com/ip/7GHFLVWR8INN24

https://walmart.com/ip/3WV6PNTBC4HL
25
https://walmart.com/ip/3M2QJUUJP

https://walmart.com/ip/5FVEVX4FM5B9https://walmart.com/ip/1VSXX7X2TFZS
3025
71
https://walmart.com/ip/4Y06I6J3EYKZ


7230
https://walmart.com/ip/6VTLWECQ5KFC30
73
https://walmart.com/ip/4619IAHLTFP4
10

74
75
20
76
https://walmart.com/ip/1AUYFEBB568O
30
29
77
https://walmart.com/ip/3QMA3TJ87OUR
https://walmart.com/ip/2FOM4Q3P075K
78

https://walmart.com/ip/2Y5HNCSBLB54https://walmart.com/ip/1M8P0MOOE1GB

29
15
79
https://walmart.com/ip/4CGVX8C8R5Q9
80
https://walmart.com/ip/1ZTY3SXRN8IO
30
81
https://walmart.com/ip/5WZOEUCQMNPC
30
82
https://walmart.com/ip/15BXMHDZ4Y8J
Requesting URL: https://walmart.com/ip/2RWTCU9KSNJ9Requesting URL: https://walmart.com/ip/2S5O8OKO03RL

30
83
https://walmart.com/ip/6IE44W82BWSK
25
84
https://walmart.com/ip/0VMIR4IKE5QS
Requesting URL: https://walmart.com/ip/76A87ZBVQYRV
Requesting URL: https://walmart.com/ip/4P8S367KE1JP
Requesting URL: https://walmart.com/ip/5BKCRKW980TE
Requesting URL: https://walmart.com/ip/3UTHU1T7755E
Requesting URL: https://walm

30Requesting URL: https://walmart.com/ip/7AHXG0SD7RPD30
117
https://walmart.com/ip/4SKC2PDR4JU0


11830
29https://walmart.com/ip/19N5ASTW9BOR

119

120
https://walmart.com/ip/6VFYK7BABA01
Getting data from DOM ...Closing Chrome ...
15
https://walmart.com/ip/2FZX8EXTTSJL121


20https://walmart.com/ip/71TE33OUS83P
122
https://walmart.com/ip/1XKVEVIIY6Y7

1Requesting URL: https://walmart.com/ip/65YKWI90RMW6Requesting URL: https://walmart.com/ip/5HMLJQWYD9PGWebpage found ...Requesting URL: https://walmart.com/ip/31AJIX2E336A


Requesting URL: https://walmart.com/ip/6YECO2QTGFPC
'NoneType' object has no attribute 'text'Requesting URL: https://walmart.com/ip/715UK7Y327Z6

Getting data from DOM ...
30
123

124

Closing Chrome ...1

Requesting URL: https://walmart.com/ip/4QU02TPZAA1W
'NoneType' object has no attribute 'text'https://walmart.com/ip/13WCIG7ZE7RYhttps://walmart.com/ip/343EW6BIU181


125
25https://walmart.com/ip/76FDD6Y4HM3R

126Getting data from DOM ...
1
'NoneType' object has no 

164
Getting data from DOM ...https://walmart.com/ip/4MH05MRJ0P21
30
165
https://walmart.com/ip/79LI1A6JM1RC

Closing Chrome ...
1
'NoneType' object has no attribute 'text'
166
Webpage found ...https://walmart.com/ip/7KGKXCK56Q4M

Closing Chrome ...
20
167
https://walmart.com/ip/1XN96MAJZSIMGetting data from DOM ...
30
168
https://walmart.com/ip/6G28HVGUEO2U

5
169
https://walmart.com/ip/0WKT0MEVO4BU
Closing Chrome ...Getting data from DOM ...

1
'NoneType' object has no attribute 'text'Closing Chrome ...
170

15https://walmart.com/ip/2SWU6H7ODH6D

171
https://walmart.com/ip/4JQNMXEX5FXC
Getting data from DOM ...25

172
Webpage found ...
https://walmart.com/ip/4NF5L5HYCNKI
Getting data from DOM ...
301

'NoneType' object has no attribute 'text'173

https://walmart.com/ip/7K18MQ7FI59G
174
https://walmart.com/ip/3XVH7IKA83ZO
Webpage found ...
Webpage found ...
1
175
https://walmart.com/ip/7C4JFLZIGFKF3030
176Requesting URL: https://walmart.com/ip/60K3G7SIGXAHRequesting URL: https://walmar

Getting data from DOM ...30
216
https://walmart.com/ip/7IH3EDY15ESJ
Requesting URL: https://walmart.com/ip/1EXR3Z7EW5UA
Getting data from DOM ...
30
Requesting URL: https://walmart.com/ip/15S3AJGQTJACRequesting URL: https://walmart.com/ip/63T63J7XMZYG


Closing Chrome ...
217
https://walmart.com/ip/5YF4G2VWJVIC
Closing Chrome ...Closing Chrome ...Requesting URL: https://walmart.com/ip/3PKC9MKMTKEQ


15
218
https://walmart.com/ip/73EVJPZD5EO4Requesting URL: https://walmart.com/ip/2VWQMLVBZ0SN

Requesting URL: https://walmart.com/ip/2GKFDS6IEMHA
Getting data from DOM ...
Getting data from DOM ...
Closing Chrome ...
Getting data from DOM ...
Closing Chrome ...25
219
https://walmart.com/ip/6X7DG3Q7LM74

Closing Chrome ...Getting data from DOM ...
Webpage found ...

Closing Chrome ...
Closing Chrome ...
Getting data from DOM ...
Getting data from DOM ...
Getting data from DOM ...
Getting data from DOM ...
Closing Chrome ...
30
220
https://walmart.com/ip/5DL0R0QCGB8I1

221
https://walmart.co

...

Webpage found ...
Webpage found ...
Closing Chrome ...
Closing Chrome ...
Closing Chrome ...
Closing Chrome ...
Getting data from DOM ...
Getting data from DOM ...
Getting data from DOM ...
Closing Chrome ...Getting data from DOM ...

Closing Chrome ...
Closing Chrome ...Getting data from DOM ...
Getting data from DOM ...Closing Chrome ...


Webpage found ...
Requesting URL: https://walmart.com/ip/3W2HLABJP6JK
15
230510

Getting data from DOM ...
https://walmart.com/ip/42QORYDY02X2
'str' object has no attribute 'text'
Getting data from DOM ...2306
https://walmart.com/ip/5VU1GZITL8XX

Closing Chrome ...Closing Chrome ...

Closing Chrome ...
Webpage found ...
Getting data from DOM ...
Getting data from DOM ...
Getting data from DOM ...Webpage found ...

1
'NoneType' object has no attribute 'text'
2307
https://walmart.com/ip/7DGBRSJ7VZR4
20
2308
https://walmart.com/ip/6GXMUKDGR6OZ
Closing Chrome ...
Closing Chrome ...
Getting data from DOM ...
1
'NoneType' object has no attribute 'text'
2

30
2355
https://walmart.com/ip/1JCJRX7BNO2Z
30
2356
Webpage found ...
https://walmart.com/ip/386KMR6MZI87
Getting data from DOM ...
Webpage found ...
Webpage found ...
Requesting URL: https://walmart.com/ip/1G9TWRKOSNTLRequesting URL: https://walmart.com/ip/4WUASGM97FRO
Requesting URL: https://walmart.com/ip/2QZ259XH76EVRequesting URL: https://walmart.com/ip/6Q7J9AEZNYW1Requesting URL: https://walmart.com/ip/7EVT5L3TOB9CRequesting URL: https://walmart.com/ip/2YME3UJSXBQF


Requesting URL: https://walmart.com/ip/43GXAXULC67K
1

Closing Chrome ...

2357
https://walmart.com/ip/4EI8EKGVS7V7
Requesting URL: https://walmart.com/ip/2KEFP160IU7C
Requesting URL: https://walmart.com/ip/1JCJRX7BNO2ZWebpage found ...

Requesting URL: https://walmart.com/ip/386KMR6MZI87Requesting URL: https://walmart.com/ip/3LNQXC63DFVA

Getting data from DOM ...
29Closing Chrome ...
2358

https://walmart.com/ip/5N3F3BI9BECI
30
2359
https://walmart.com/ip/7C00TJ3H5NT3
Closing Chrome ...
Closing Chrome ...Closing Ch

Webpage found ...
Getting data from DOM ...
Webpage found ...
Getting data from DOM ...
Closing Chrome ...
30Webpage found ...30

2407
https://walmart.com/ip/7FQBCCSTBFQ9

2530
2408
2409
https://walmart.com/ip/2A2SCPNRP80H
2410
Getting data from DOM ...https://walmart.com/ip/76KGM83AETOK


Requesting URL: https://walmart.com/ip/3RMGL5OXJKZP
https://walmart.com/ip/64WAVVOT2IH0Requesting URL: https://walmart.com/ip/62FEL6JPL3NTRequesting URL: https://walmart.com/ip/1DSOI5Z684BK


Webpage found ...
1
Requesting URL: https://walmart.com/ip/1XSFEBHKK1K3
Requesting URL: https://walmart.com/ip/4Z412OJ7PEFR
2411
Requesting URL: https://walmart.com/ip/41SZOZ0E2Y5Xhttps://walmart.com/ip/3CSTDEWFQF5X

Requesting URL: https://walmart.com/ip/2GWIEH4PXKMZRequesting URL: https://walmart.com/ip/49YU1KC8GS5V

Requesting URL: https://walmart.com/ip/30O5DXJL6FVS
Webpage found ...Webpage found ...Webpage found ...


Closing Chrome ...
Closing Chrome ...Closing Chrome ...

Getting data from DOM ...Closing 

Requesting URL: https://walmart.com/ip/6NUMZ8DAK8KRRequesting URL: https://walmart.com/ip/5XUZ5Z3MDHY8'NoneType' object has no attribute 'text'



2451
Webpage found ...
Webpage found ...Requesting URL: https://walmart.com/ip/6JSSHN0XB9I8

https://walmart.com/ip/1QX2GICBWQMM
Requesting URL: https://walmart.com/ip/6DENI6QR4QRS
Closing Chrome ...
251

24522453
https://walmart.com/ip/1PRH21OO2VRC

https://walmart.com/ip/1VR1WX1KXWGG
Getting data from DOM ...
29Closing Chrome ...

Closing Chrome ...2454
https://walmart.com/ip/5D8V8QXWG5PU
Closing Chrome ...Closing Chrome ...

Closing Chrome ...

Getting data from DOM ...
30Getting data from DOM ...

2455
Getting data from DOM ...
https://walmart.com/ip/7DIMCQ5KOT49
Getting data from DOM ...Webpage found ...

Getting data from DOM ...
Webpage found ...30

2456
https://walmart.com/ip/6YKLHQ5G1KJQ
Webpage found ...
25
30
2457
2458
https://walmart.com/ip/25USCGS55X2S
https://walmart.com/ip/18OM23JGBZO4
29
Webpage found ...
Requesting URL: http

DoneAndNotDoneFutures(done={<Future at 0x7f8d4b060050 state=finished returned NoneType>, <Future at 0x7f8d656e0090 state=finished returned NoneType>, <Future at 0x7f8d65d800d0 state=finished returned NoneType>, <Future at 0x7f8d685800d0 state=finished returned NoneType>, <Future at 0x7f8d6a4e00d0 state=finished returned NoneType>, <Future at 0x7f8d65d80110 state=finished returned NoneType>, <Future at 0x7f8d50ee0150 state=finished returned NoneType>, <Future at 0x7f8d6a4e0150 state=finished returned NoneType>, <Future at 0x7f8d65e60190 state=finished returned NoneType>, <Future at 0x7f8d68580190 state=finished returned NoneType>, <Future at 0x7f8d65d801d0 state=finished returned NoneType>, <Future at 0x7f8d656e01d0 state=finished returned NoneType>, <Future at 0x7f8d65d80210 state=finished returned NoneType>, <Future at 0x7f8d65e60210 state=finished returned NoneType>, <Future at 0x7f8d656e0210 state=finished returned NoneType>, <Future at 0x7f8d65e60250 state=finished returned NoneTyp

for pg in range(len(url_sets)):
    # number of pages per category
    top_n= ["1","2","3","4","5","6","7","8","9","10"]
    url_category=url_sets[pg]
    print("Category:",categories[pg])
    final_results = []
    for i_1 in range(len(top_n)):
        print("Page number within category:",i_1)
        url_cat=url_category+"?page="+top_n[i_1]
        driver= webdriver.Chrome(executable_path='/Users/bhaskar/Documents/chromedriver.exe')
        driver.get(url_cat)
        body_cat = driver.find_element_by_tag_name("body").get_attribute("innerHTML")
        driver.quit()
        soupBody_cat = BeautifulSoup(body_cat)
        
        
        for tmp in soupBody_cat.find_all('div', {'class':'search-result-gridview-item-wrapper'}):

            final_results.append(tmp['data-id'])
    codelist=final_results
    print("Total number of prods:",len(codelist))
    # base URL for product page
    url1= "https://walmart.com/ip"

    # Data Headers
    WLMTData = [["Product_code","Product_name", "Product_description","Product_URL",
                 "Breadcrumb_parent","Breadcrumb_active",
                "Product_price", "Rating_Value","Rating_Count","Recommended_Prods"]]
    for i in range(len(codelist)):
        #creating a list without the place taken in the first loop
        print(i)
        item_wlmt=codelist[i]
        url2=url1+"/"+item_wlmt
        #print(url2)
        
        try:
            driver= webdriver.Chrome(executable_path='C:/Drivers/chromedriver.exe') # Chrome driver is being used.
            print ("Requesting URL: " + url2)

            driver.get(url2)   # URL requested in browser.
            print ("Webpage found ...")
            time.sleep(5)
            # Find the document body and get its inner HTML for processing in BeautifulSoup parser.
            body = driver.find_element_by_tag_name("body").get_attribute("innerHTML")
            print("Closing Chrome ...") # No more usage needed.
            driver.quit() 				# Browser Closed.

            print("Getting data from DOM ...")
            soupBody = BeautifulSoup(body) # Parse the inner HTML using BeautifulSoup
            #print(soupBody)

            h1ProductName = soupBody.find("h1", {"class": "prod-ProductTitle prod-productTitle-buyBox font-bold"})
            divProductDesc = soupBody.find("div", {"class": "about-desc about-product-description xs-margin-top"})
            liProductBreadcrumb_parent = soupBody.find("li", {"data-automation-id": "breadcrumb-item-0"})
            liProductBreadcrumb_active = soupBody.find("li", {"class": "breadcrumb active"})
            spanProductPrice = soupBody.find("span", {"class": "price-group"})
            spanProductRating = soupBody.find("span", {"itemprop": "ratingValue"})
            spanProductRating_count = soupBody.find("span", {"class": "stars-reviews-count-node"})
            
            ################# exceptions #########################
            if divProductDesc is None:
                divProductDesc="Not Available"
            else:
                divProductDesc=divProductDesc
                
            if liProductBreadcrumb_parent is None:
                liProductBreadcrumb_parent="Not Available"
            else:
                liProductBreadcrumb_parent=liProductBreadcrumb_parent
                
            if liProductBreadcrumb_active is None:
                liProductBreadcrumb_active="Not Available"
            else:
                liProductBreadcrumb_active=liProductBreadcrumb_active
                
            if spanProductPrice is None:
                spanProductPrice="NA"
            else:
                spanProductPrice=spanProductPrice

            if spanProductRating is None or spanProductRating_count is None:
                spanProductRating=0.0
                spanProductRating_count="0 ratings"
            #print(spanProductBreadcrumb_active.text)
            else:
                spanProductRating=spanProductRating.text
                spanProductRating_count=spanProductRating_count.text


            ### Recommended Products
            reco_prods=[]
            for tmp in soupBody.find_all('a', {'class':'tile-link-overlay u-focusTile'}):
                reco_prods.append(tmp['data-product-id'])

            if len(reco_prods)==0:
                reco_prods=["Not available"]
            else:
                reco_prods=reco_prods
            print(len(reco_prods))
            WLMTData.append([codelist[i],h1ProductName.text, divProductDesc.text,url2,
                            liProductBreadcrumb_parent.text,liProductBreadcrumb_active.text,
                            spanProductPrice.text,spanProductRating,spanProductRating_count,reco_prods])




            except Exception as e:
                print (str(e))

#WLMTData        

In [19]:
df=pd.DataFrame(WLMTData)
df.columns = df.iloc[0]
df=df.drop(df.index[0])
df

,Product_code,Product_name,Product_description,Product_URL,Breadcrumb_parent,Breadcrumb_active,Product_price,Rating_Value,Rating_Count,Recommended_Prods
1,0U3YGN4RJB8T,Oral-B Pro 7500 Power Rechargeable Electric To...,,https://walmart.com/ip/0U3YGN4RJB8T,Personal Care,/Oral-B Electric Toothbrushes,$134.94,4.8,19 ratings,[Not available]
2,53V4NM0KP2DK,Portable Electric Sonic Dental Scaler Tooth Ca...,Feature： Dental Calculus Remover: with up to ...,https://walmart.com/ip/53V4NM0KP2DK,Personal Care,/All Electric Toothbrushes,$15.99,3.0,2 ratings,"[1PM103W37PR8, 20CIOCLC3MN3, 423OJ2IA1I8H, 5A7..."
3,1SLP0ZRCUWSH,Oral-B 3D White Electric Toothbrush Replacemen...,The Oral-B Sensitive Gum Care replacement toot...,https://walmart.com/ip/1SLP0ZRCUWSH,Personal Care,/Kids Oral Care,$24.95,4.9,326 ratings,"[4AWREEP5Q8FQ, 56EHCRU3WQ84, 5HBAERP3RRY6, 4NE..."
4,0VMFUHILVSOY,Philips Sonicare Essence Rechargeable Electric...,Sonic fluid action gently pulses between the t...,https://walmart.com/ip/0VMFUHILVSOY,Personal Care,/Philips Sonicare Electric Toothbrushes,$69.97,4.5,3528 ratings,"[5KQ5I4WZ4L6V, 685Y0QDGANUW, 3LTZM31X686A, 5YX..."
5,6VLHJPUHZWEF,"Colgate Kids Battery Powered Toothbrush, Ryan'...",Great clean made fun. Your little one will loo...,https://walmart.com/ip/6VLHJPUHZWEF,Gifts & Registry,/Gift Finder,$5.00,3.5,2 ratings,"[2BVLJCB1Y00B, 0RX3EUB764IK, 4XPVRY0B04R9, 2WA..."
...,...,...,...,...,...,...,...,...,...,...
1960,56KWV9F2ZHPW,"Spring Valley Vitamin D3 Gummy, 2000 IU, 80 Ct",Spring Valley Vitamin D3 Gummy is a great chew...,https://walmart.com/ip/56KWV9F2ZHPW,Health,/Pre Workout,$4.88,4.8,21 ratings,"[3QISYVBWFFRK, 6H6PRKJR84TK, 12HKPEVS0UO8, 2ZQ..."
1961,3DQ6B3QQM6NR,"CLIF Kid Zbar Organic Granola Bars, Kids Snack...","CLIF Kid Zbar is an organic, baked whole grain...",https://walmart.com/ip/3DQ6B3QQM6NR,Sports & Outdoors,/Bike Bars,$11.28,4.8,26 ratings,"[30MKGQT3NN95, 4VT2JUPD7OVO, 5P3P0YI03K5K, 3XA..."
1962,43WEI2F3TE4X,Kodiak Cakes Chocolate Chip Oatmeal in a Cup,,https://walmart.com/ip/43WEI2F3TE4X,Health,/On-the-Go Nutrition,$5.64,4.7,3 ratings,[Not available]
1963,5UK3918K77FR,"CLIF Bar Energy Bars, Crunchy Peanut Butter, 1...",Born on a 175-mile bike ride and purposefully ...,https://walmart.com/ip/5UK3918K77FR,Food,/CLIF Bar Energy Bars and Protein Bars,$5.78,4.7,60 ratings,"[247TAONXUCAG, 2BE79P9RY6VQ, 4SLRJP1GXPXU, 0UP..."


In [20]:
# Output File for FlightsData. This file will have the data in comma separated form.
df.to_csv('personal_care.csv',index=False)